# Homework 2 submission
for course: Data Science lab 2   
by  Michael Berger   
 

Lecturer: Dr. Sharon Yalov-Handzel  
Afeka College of engineering


## 1. Use the dataset from UCI Machine Learning Repository:
"Individual household electric power consumption" for performing time series analysis.  
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

In [ ]:
! pip install ucimlrepo

In [ ]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import root_mean_squared_error,    r2_score
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer


import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, Dense, Attention, Concatenate



import matplotlib.pyplot as plt
#import tensorflow as tf
#import pytorch as pt

import os
from datetime import datetime
import statistics as stat

import plotly.express as px
from plotly.subplots import make_subplots

from ucimlrepo import fetch_ucirepo , dotdict
pd.set_option('display.precision', 3)

from pathlib import Path
import pickle 
 

%matplotlib inline

In [ ]:
sklearn.__version__

In [ ]:

fn = r"data/individual_household_electric_power_consumption.pkl"
pth = Path(fn)
if pth.exists():
    with open(fn, "rb") as fl:
        individual_household_electric_power_consumption = pickle.load(fl)
else: 
    
    individual_household_electric_power_consumption = fetch_ucirepo(id=235) 
    pth.parent.mkdir(parents=True, exist_ok=True)
    with open(fn, "wb") as fl:
        pickle.dump(individual_household_electric_power_consumption, fl)
    

In [ ]:
# data (as pandas dataframes) 
X = individual_household_electric_power_consumption.data.features 
y = individual_household_electric_power_consumption.data.targets 
  
# metadata 
print(individual_household_electric_power_consumption.metadata) 
  
# variable information 
print(individual_household_electric_power_consumption.variables) 

In [ ]:
import json
meta = json.dumps(individual_household_electric_power_consumption.metadata,indent=2)
print(meta)

In [ ]:
metadic = individual_household_electric_power_consumption.metadata

In [ ]:
type(metadic)

In [ ]:
type(metadic).__name__

In [ ]:
def mylen(o):
    if type(o)==int:
        return str(o)
    if type(o)==type(None):
        return "0"
    else:
        return len(o)

In [ ]:
def showkeys(dic, ind = 0):
    if type(dic)== dotdict:
        for k,v  in dic.items() :
            print(" "*ind , k, "|", type(v).__name__, ",", mylen(v) )
            showkeys(dic[k], ind+4)

showkeys(metadic)

In [ ]:
print(metadic["abstract"])

In [ ]:
print(metadic["additional_info"]["summary"])

In [ ]:
print(metadic["additional_info"]["variable_info"])

## 2. Perform Exploratory Data Analysis (EDA) of the dataset:

First, convert all values to numeric

In [ ]:
numcols = ['Global_active_power', 'Global_reactive_power',
           'Voltage', 'Global_intensity', 'Sub_metering_1', 
           'Sub_metering_2', 'Sub_metering_3']

for c in numcols:
    X[c] = pd.to_numeric(X[c], errors = "coerce") 

combine date and time into single col datetime

In [ ]:
dt = X["Date"] + " " + X["Time"]
X["datetime"] = pd.to_datetime(dt, dayfirst=True)

### a.  Visualize time series trends

In [ ]:
def summary(df):
    nonnans = df.shape[0] - df.isna().sum()
    nonnansPrc = (nonnans / df.shape[0] * 100).apply("{0:.2f}%".format)
    sam = df.sample(1, random_state=42).squeeze()
    res = pd.DataFrame([sam.index, df.dtypes.astype(str), nonnans,
                        nonnansPrc, df.nunique(), sam]).transpose()
    res.columns = ["Column", "data type", "non-null values", 
                   "non-null values %", "unique values", "example"]
    res.sort_values("unique values",ascending=False, inplace=True)
    return res

In [ ]:
X.head()

In [ ]:

pd.options.display.float_format = '{:,.4f}'.format
d = X.describe()
d

In [ ]:
summary(X)

In [ ]:
X

In [ ]:
df = X.set_index('datetime').drop(["Date","Time"],axis=1)

In [ ]:
df = df.fillna(0)
df

### Try: Seasonal decompose

In [ ]:
test = df['Global_active_power'].head(10000)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose


# Decomposing the time series
result = seasonal_decompose(test, model='additive', period=7) #, extrapolate_trend='freq')

In [ ]:
from statsmodels.tsa.arima import 

In [ ]:
result.trend.shape

In [ ]:
# Plotting the decomposed time series
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(14, 12))

# Original data
ax1.plot(test, '-', label='Original Data')
ax1.set_title('Original Time Series')
ax1.grid(True)

# Trend component
ax2.plot(result.trend, '-', label='Trend Component')
ax2.set_title('Trend Component')
ax2.grid(True)

# Seasonal component
ax3.plot(result.seasonal, '-', label='Seasonal Component')
ax3.set_title('Seasonal Component')
ax3.grid(True)

# Residual component
ax4.plot(result.resid, '-', label='Residual Component')
ax4.set_title('Residual Component')
ax4.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
Xs = X.sample(int(10e3))

In [ ]:
Xs.shape

In [ ]:
X[X.datetime == X.loc[124076,"datetime"]]

In [ ]:
px.histogram(Xs, x = 'Global_active_power')

In [ ]:
Xss = Xs.sort_values('datetime')

In [ ]:
Xss.reset_index(inplace=True)

In [ ]:
px.line(Xss, x='datetime', y = 'Global_reactive_power',)

### Trying to apply FFT

In [ ]:
import librosa
import librosa.display

In [ ]:
X['Global_active_power'].dropna()

In [ ]:
s,l = 1e5, 1e5
e = s+l
test = X.loc[s:e-1]
test.shape

In [ ]:
test.index

In [ ]:
test.loc['Gap_diff']  = test['Global_active_power'].diff()

In [ ]:
test

In [ ]:
px.line(test, x='datetime', y = 'Global_active_power', hover_data=[test.index])

In [ ]:
60*24

In [ ]:
px.line(test, x='datetime', y = 'Gap_diff', hover_data=[test.index])

In [ ]:
X['Gap_diff']  = X['Global_active_power'].diff()

In [ ]:
X

In [ ]:
audio = np.array(X['Gap_diff'].fillna(0)) 
audio

In [ ]:
audio.shape

In [ ]:
audio.diff()

In [ ]:
px.histogram(audio)

In [ ]:
np.mean(audio)

In [ ]:
2400/14/2/24

In [ ]:
W = np.ones(Wsize)/Wsize

In [ ]:
W

In [ ]:
Wsize = 1200
ma = np.convolve(audio, np.ones(Wsize)/Wsize, mode='same',)

In [ ]:
sd = np.sqrt(np.convolve((audio - ma)**2, np.ones(Wsize)/Wsize, mode='same',) +1) 

In [ ]:
np.min(sd)

In [ ]:
audio[audio == ma]

In [ ]:
audio.shape , ma.shape, sd.shape

In [ ]:
sd[sd==0]

In [ ]:
2049280 - 2049221

In [ ]:
audio2 = (audio) -ma


In [ ]:
import plotly.graph_objects as go 

In [ ]:
test

In [ ]:
#fig = px.line(test, x='datetime', y = test.index)
#fig.add_scatter(test,  y = 'Global_active_power', hover_data=[test.index])
#fig.add_scatter(y = ma, x = test.datetime)
#fig.add_scatter(y = sd, x = test.datetime)

In [ ]:
px.line(audio2)

In [ ]:
audio[~np.isfinite(audio)]

In [ ]:
audio2.shape

In [ ]:
60*60*24*7

In [ ]:
2**19

In [ ]:
pow = 9
spec_params = {"n_fft": 2**(pow), "hop_length": 2**(pow-4), "window":"blackman" }
spec_params

In [ ]:
D = np.abs(librosa.stft(y=audio2, **spec_params,))
DB = librosa.amplitude_to_db(D, ref=np.max)

DB.shape


Hz Hertz  = samples per second  
SHz Shmerz = samples per day  


my signal period = 1/1440 day  
my sampling freq = 60 min * 24 hrs = 1440 SHz  



In [ ]:
Fs = 60*24
nfft = 512
f = range(0,nfft/2-1)*Fs/nfft; 

In [ ]:
f = np.arange(0,int(nfft/2)-1)*(Fs/nfft)

In [ ]:
f

In [ ]:
f.shape

In [ ]:
DB[:,0:1000]

In [ ]:
import scipy
scipy.signal.get_window()

In [ ]:
fs= 4096 # 86400 # 44100

In [ ]:
DL = np.log10(D)

In [ ]:
#px.imshow(DB[:,0:5000],aspect='auto', origin='lower')

In [ ]:
fig = px.imshow(DB[:,5000:9000], aspect='auto', origin='lower') #, extent=[80,120,32,0] )
fig.update_yaxes(range=[0, f[-1]], autorange=False)

### b. Check for seasonality and cyclical patterns

- Analyze distribution of power consumption
- Identify and handle missing values or outliers

## 3. Implement a linear regression model to predict power consumption for the last three time periods:

### a. Split the data into training and testing sets

In [ ]:
dummyScaler = FunctionTransformer(lambda a: a, lambda a: a) 

In [ ]:
n_sampls = 10000
epochs = 5
verbose = 1

In [ ]:
Xs = X.loc[0:n_sampls,:]

In [ ]:
Xs = Xs.set_index("datetime")

### b. Prepare features (consider lag variables, time-based features) 
features: 
year
month 
week
day
hour
every variable + 10 of it's lags 

y's: 
Global_active_power

In [ ]:
# resample the data to fill the NaN values
df= Xs["Global_active_power"].resample("1min").mean().ffill().to_frame()
df

In [ ]:
df['year'] = df.index.year 
df['month'] = df.index.month 
df['day'] = df.index.day 
df['hour'] = df.index.hour 
df['minute'] = df.index.minute 
df

In [ ]:
for i in range(0,6):
    lag = 3**i
    df[f"Global_active_power.L{lag}"] = df["Global_active_power"].shift(lag)

In [ ]:
df.dropna(inplace = True)
df

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
y = df[target]
X = df.drop(columns= target)                                             
#Split the data into train and test sets
cutoff = int(len(X) * 0.8)
X_train, y_train = X.iloc[:cutoff], y.iloc[:cutoff]
X_test, y_test = X.iloc[cutoff:], y.iloc[cutoff:]

In [ ]:
X_train

### c. Train the model and make predictions

In [ ]:
model = LinearRegression()
res = model.fit(X_train, y_train)


In [ ]:
dir(res)

In [ ]:
df_pred_test = pd.DataFrame(
             {
             "y_test": y_test,
             "y_pred": model.predict(X_test)
             }
)
df_pred_test

In [ ]:
fig = px.line(df_pred_test, labels= {"value": "Close Price"}, 
              title = "Linear Regression Model: Actual Prices vs. Predicted Prices.")
fig.show()

## 4. Evaluate the linear regression model using appropriate metrics:
- Mean Absolute Error (MAE)
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- R-squared (R²) value

In [ ]:
experiments = pd.DataFrame(columns=['model','data'])

In [ ]:
def pipeline(model, x_train, y_train, x_test, y_test, model_args={}):

    # Train the model
    print(f"training model, please wait...")
    model.fit(x_train, y_train, **model_args) # , epochs=epochs, validation_split= .1, verbose=0)
    
    print(f"evaluating model, please wait...")
    y_trn_pred = model.predict(x_train)    
    y_pred = model.predict(x_test)
    
   
    #df_pred_test = pd.DataFrame({"y_test": y_test,"y_pred": y_pred})
   
    res = {}
    print(y_train.shape, y_trn_pred.shape)
    # Evaluate the model on train set
    y_train = y_train.squeeze()
    y_trn_pred = y_trn_pred.squeeze()
    print(y_train.shape, y_trn_pred.shape)
    
    res["trn_MAE"] = [mean_absolute_error(y_train, y_trn_pred)]
    res["trn_MSE"] = [mean_squared_error(y_train, y_trn_pred)]
    res["trn_RMSE"] = [root_mean_squared_error(y_train, y_trn_pred)]
    res["trn_Rsq"] = [r2_score(y_train, y_trn_pred)]
    
    # Evaluate the model on test set
    y_test = y_test.squeeze()
    y_pred = y_pred.squeeze()
    print(y_test.shape, y_pred.shape)
    
    res["tst_MAE"] = [mean_absolute_error(y_test, y_pred)]
    res["tst_MSE"] = [mean_squared_error(y_test, y_pred)]
    res["tst_RMSE"] = [root_mean_squared_error(y_test, y_pred)]
    res["tst_Rsq"] = [r2_score(y_test, y_pred)]
    print("done")
    return model, res
  

In [ ]:
def report(result, model_label, data_label):
    a = result
    a["model"] = model_label
    a["data"] = data_label
    log_df = pd.DataFrame(a)    
    global experiments 

    if((experiments.shape[0]>0) and  
       model_label in experiments.model.unique() and  
       data_label in experiments.data.unique()):
        import warnings
        warnings.warn(f"model label {model_label} and data label '{data_label}' already exist. replacing.")
        #drop(fit_runs, 'data', label)
    
    experiments = pd.concat([experiments, log_df])

    metrics = list(result.keys())
    metrics.sort()
    
    log_df_melted = experiments.melt(id_vars=['model','data'], 
                            value_vars= metrics, #['accuracy', 'loss'], 
                            var_name='metric', 
                            value_name='value')
    
    rx = r"(?P<set>[trns]+_)?(?P<metric>[A-Za-z]+)"
    nspl = log_df_melted.metric.str.extract(rx)
    log_df_melted.drop('metric',axis=1, inplace=True)
    log_df_melted = log_df_melted.join(nspl)    
    log_df_melted.loc[log_df_melted.set == "trn_", "set"] = "trn"
    #log_df_melted.loc[log_df_melted.set == "val_", "set"] = "val"
    log_df_melted.loc[log_df_melted.set == "tst_", "set"] = "tst"

    fig = px.bar(log_df_melted, x='model', y='value', title= "use mouse to zoom",
                  color='set', facet_row='metric', height=600, barmode = 'group',
                  facet_col="data", category_orders={"set": ["trn","val","tst"]}
                 )

    #fig.update_yaxes(matches='facet_row')
    return fig

In [ ]:
X_train

In [ ]:
model = DummyRegressor(strategy='mean')

In [ ]:
model, res = pipeline(model, X_train, y_train, X_test, y_test)

report(res, "Baseline", "basic")


In [ ]:
model = LinearRegression()

In [ ]:
model, res = pipeline(model, X_train, y_train, X_test, y_test)
report(res, "LinearRegression", "basic")

In [ ]:
experiments

In [ ]:

#mean absolute error


y_pred_baseline = [y_train.mean()] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)
print("Mean Global_active_power:", round(y_train.mean(), 2))
print("Baseline MAE:", round(mae_baseline, 4))
# Mean Close Prices: 391.22
#Baseline MAE: 91.89

In [ ]:
training_mae = mean_absolute_error(y_train, model.predict(X_train))
test_mae = mean_absolute_error(y_test, model.predict(X_test))
print("Training MAE:", round(training_mae, 2))
print("Test MAE:", round(test_mae, 2))

## 5. Implement a Recurrent Neural Network (RNN) for power consumption prediction:

### a. Preprocess data for RNN input

In [ ]:
df.shape

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
y = df[[target]]
X = df.drop(columns= target)                     

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

#Split the data into train and test sets
cutoff = int(len(X) * 0.8)
X_train, y_train = X[:cutoff], y[:cutoff]
X_test, y_test = X[cutoff:], y[cutoff:]

In [ ]:
X.shape

### b. Design and train the RNN model

In [ ]:
sequence_length = X_train.shape[1]

# Build an RNN model
simpleRnn_model = Sequential([
    SimpleRNN(units=32, activation='relu', input_shape=(sequence_length, 1)),
    Dense(units=1)
])

simpleRnn_model.compile(optimizer='adam', loss='mse')
model_args = {"epochs":epochs, "batch_size":16}

In [ ]:
# Train the model
simpleRnn_model, res = pipeline(simpleRnn_model, X_train, y_train, X_test, y_test, model_args)



- Make predictions and visualize results  

 
 
 TODO: visualise the time series with predictions 

In [ ]:
report(res, "SimpleRNN", "scaled")

- Compare performance metrics with linear regression

## 6. Implement Long Short-Term Memory (LSTM) for power consumption prediction:
 

### a. Preprocess data for LSTM input

In [ ]:
# Split the data into feature and target
target = "Global_active_power"
y = df[[target]]
X = df.drop(columns= target)                     

# Normalize the dataset
# Reshape input to be [samples, time steps, features] which is required for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X).reshape(X.shape[0], X.shape[1], 1)
y = scaler.fit_transform(y).reshape(y.shape[0], y.shape[1], 1)

time_step = X.shape[1]

#Split the data into train and test sets
cutoff = int(len(X) * 0.8)
X_train, y_train = X[:cutoff], y[:cutoff]
X_test, y_test = X[cutoff:], y[cutoff:]

In [ ]:
X_train.shape, y_train.shape

### b. Design and train the LSTM model

In [ ]:
# Create and fit the LSTM network
LSTM_model = Sequential()
LSTM_model.add(Input((time_step, 1)))
LSTM_model.add(LSTM(50, return_sequences=True ))
LSTM_model.add(LSTM(50, return_sequences=False))
LSTM_model.add(Dense(1))

LSTM_model.compile(optimizer='adam', loss='mean_squared_error')

model_args = {"epochs":epochs, "batch_size":32, "verbose":verbose}

In [ ]:
model_args

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
LSTM_model, res = pipeline(LSTM_model, X_train, y_train, X_test, y_test, model_args)


In [ ]:

# # Make predictions
# train_predict = model.predict(X)

# # Invert predictions
# train_predict = scaler.inverse_transform(train_predict)
# Y = scaler.inverse_transform([Y])

### c. Make predictions and visualize results

### d. Compare performance metrics with previous models

In [ ]:
report(res, "LSTM", "scaled")

## 7. Implement an LSTM model with an Attention layer for power consumption prediction:

### a. Design and train the LSTM model with Attention

In [ ]:
# Attention-based Encoder-Decoder model
encoder_input = Input(shape=(time_step, 1))
encoder_lstm = LSTM(32, return_sequences=True)(encoder_input)

decoder_input = Input(shape=(time_step-2, 1))
decoder_lstm = LSTM(32, return_sequences=True)(decoder_input)

attention = Attention()([decoder_lstm, encoder_lstm])
combined = Concatenate(axis=-1)([decoder_lstm, attention])

output = Dense(1)(combined)

attention_model = Model(inputs=[encoder_input, decoder_input], outputs=output)
attention_model.compile(optimizer='adam', loss='mse')





In [ ]:
wat = attention_model.fit([X_train, X_train[:, :time_step-2]], y_train, **model_args) 

In [ ]:
ypred = attention_model.predict([X_test, X_test[:, :time_step-2]])

In [ ]:
ypred.shape

In [ ]:
ytrnpred = attention_model.predict([X_train, X_train[:, :time_step-2]])

In [ ]:

ytrnpred.shape

In [ ]:
attention_model

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(attention_model)

In [ ]:
attention_model, res = pipeline(attention_model, 
                                [X_train, X_train[:, :time_step-2]], y_train, 
                                [X_test, X_test[:, :time_step-2]], y_test, model_args)


### b. Make predictions and visualize results

### c. Compare performance metrics with previous models

### d. Analyze the Attention weights to interpret model focus

## 8. Data augmentation experiment:

- Modify up to 10% of the dataset to potentially improve prediction results
- Retrain and evaluate all three models (RNN, LSTM, LSTM with Attention)
- Compare the impact of data changes on each model's performance
## 9. Data reduction experiment:

- Remove up to 10% of the data randomly
- Retrain and evaluate all three models
- Compare how data reduction affects each model's performance
## 10. Data resolution experiment:

- Reduce the time resolution of the data by 50% (e.g., from minute-level to 2-minute
intervals)
- Retrain and evaluate all three models
- Analyze how changes in data resolution impact each model's performance
## 11. Conclusion and insights:

- Summarize findings from all experiments
- Discuss which model performed best under different conditions
- Provide insights on the dataset's characteristics and their impact on model performance